# Exploring ICS ATT&CK

## Query ATT&CK

### Import TAXII Libraries
ATT&CK users can use the initial Server class to instantiate a server object pointing to the framework’s public TAXII server URL https://cti-taxii.mitre.org/taxii/

In [1]:
from taxii2client.v20 import Server

In [2]:
server = Server("https://cti-taxii.mitre.org/taxii/")

Available API Roots can be referenced from the server object. API Roots are logical groupings of TAXII Channels and Collections and can be thought of as instances of the TAXII API available at different URLs, where each API Root is the “root” URL of that particular instance of the TAXII API:

In [3]:
server.api_roots

In [4]:
api_root = server.api_roots[0]

### Explore ATT&CK TAXII Collections
The **collections** attribute can then be used and get more information about them via their respective available properties:

In [5]:
api_root.collections

In [6]:
for collection in api_root.collections:
    print(collection.title, "->", collection.description)

Enterprise ATT&CK -> This data collection holds STIX objects from Enterprise ATT&CK
PRE-ATT&CK -> This data collection holds STIX objects from PRE-ATT&CK
Mobile ATT&CK -> This data collection holds STIX objects from Mobile ATT&CK
ICS ATT&CK -> This data collection holds STIX objects from ICS ATT&CK


In [7]:
api_root.collections[3].title

'ICS ATT&CK'

In [8]:
api_root.collections[3].id

'02c3ef24-9cd4-48f3-a99f-b74ce24f1d34'

## Set ICS ATT&CK TAXII Collection ID Variable

In [9]:
ICS_ATTACK = "02c3ef24-9cd4-48f3-a99f-b74ce24f1d34"

## Initialize TAXII Collection Sources
According to [STIX2 docs](https://stix2.readthedocs.io/en/latest/index.html), the [TAXIICollectionSource API](https://stix2.readthedocs.io/en/latest/api/datastore/stix2.datastore.taxii.html#stix2.datastore.taxii.TAXIICollectionSource) provides an interface for searching/retrieving STIX objects from a local/remote TAXII Collection endpoint. In our case, we are pointing to our ATT&CK TAXII Collection instances (https://cti-taxii.mitre.org/stix/collections/<Collection ID>)

In [10]:
from stix2 import TAXIICollectionSource, Filter
from taxii2client.v20 import Collection

In [11]:
ATTACK_STIX_COLLECTIONS = "https://cti-taxii.mitre.org/stix/collections/"
ICS_COLLECTION = Collection(ATTACK_STIX_COLLECTIONS + ICS_ATTACK + "/")
TC_ICS_SOURCE = TAXIICollectionSource(ICS_COLLECTION)

## Retrieve all ICS Techniques
Now that we can query the ICS ATT&CK TAXIICollection. We can use the query method and a set of filter to retrieve STIX objects of type "attack-pattern" -> "Techniques"

In [12]:
ICS_TECHNIQUES = TC_ICS_SOURCE.query(Filter("type", "=", "attack-pattern"))
print(ICS_TECHNIQUES[0])

{
    "type": "attack-pattern",
    "id": "attack-pattern--2900bbd8-308a-4274-b074-5b8bde8347bc",
    "created_by_ref": "identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5",
    "created": "2020-05-21T17:43:26.506Z",
    "modified": "2020-05-21T17:43:26.506Z",
    "name": "Alarm Suppression",
    "description": "Adversaries may target protection function alarms to prevent them from notifying operators of critical conditions. Alarm messages may be a part of an overall reporting system and of particular interest for adversaries. Disruption of the alarm system does not imply the disruption of the reporting system as a whole. \n\nIn the Maroochy Attack, the adversary suppressed alarm reporting to the central computer. (Citation: Maroochy - MITRE - 200808)\n\nA Secura presentation on targeting OT notes a dual fold goal for adversaries attempting alarm suppression: prevent outgoing alarms from being raised and prevent incoming alarms from being responded to. (Citation: References - Secura - 2019

In [13]:
for TECHNIQUE in ICS_TECHNIQUES:
    print(TECHNIQUE['external_references'][0]['external_id'], "--", TECHNIQUE['name'])

T0878 -- Alarm Suppression
T0806 -- Brute Force I/O
T0808 -- Control Device Identification
T0812 -- Default Credentials
T0870 -- Detect Program State
T0819 -- Exploit Public-Facing Application
T0874 -- Hooking
T0825 -- Location Identification
T0829 -- Loss of View
T0849 -- Masquerading
T0801 -- Monitor Process State
T0843 -- Program Download
T0846 -- Remote System Discovery
T0852 -- Screen Capture
T0856 -- Spoof Reporting Message
T0855 -- Unauthorized Command Message
T0803 -- Block Command Message
T0807 -- Command-Line Interface
T0809 -- Data Destruction
T0814 -- Denial of Service
T0817 -- Drive-by Compromise
T0866 -- Exploitation of Remote Services
T0824 -- I/O Module Discovery
T0827 -- Loss of Control
T0835 -- Manipulate I/O Image
T0833 -- Modify Control Logic
T0841 -- Network Service Scanning
T0845 -- Program Upload
T0848 -- Rogue Master Device
T0854 -- Serial Connection Enumeration
T0862 -- Supply Chain Compromise
T0858 -- Utilize/Change Operating Mode
T0804 -- Block Reporting Mess

## ICS ATT&CK Available in attackcti 0.3.4.3
Reference: https://pypi.org/project/attackcti/

In [14]:
from attackcti import attack_client
lift = attack_client()

ICS_TECHNIQUES = lift.get_ics_techniques()
print("Techniques Count:",len(ICS_TECHNIQUES))

Techniques Count: 81


In [15]:
print(ICS_TECHNIQUES[0])

{
    "type": "attack-pattern",
    "id": "attack-pattern--2900bbd8-308a-4274-b074-5b8bde8347bc",
    "created_by_ref": "identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5",
    "created": "2020-05-21T17:43:26.506Z",
    "modified": "2020-05-21T17:43:26.506Z",
    "name": "Alarm Suppression",
    "description": "Adversaries may target protection function alarms to prevent them from notifying operators of critical conditions. Alarm messages may be a part of an overall reporting system and of particular interest for adversaries. Disruption of the alarm system does not imply the disruption of the reporting system as a whole. \n\nIn the Maroochy Attack, the adversary suppressed alarm reporting to the central computer. (Citation: Maroochy - MITRE - 200808)\n\nA Secura presentation on targeting OT notes a dual fold goal for adversaries attempting alarm suppression: prevent outgoing alarms from being raised and prevent incoming alarms from being responded to. (Citation: References - Secura - 2019

## Get All Data Sources Mapped to ICS ATT&CK Techniques

In [16]:
ICS_DATA_SOURCES = []
for TECHNIQUE in ICS_TECHNIQUES:
    if 'x_mitre_data_sources' in TECHNIQUE.keys():
        for DS in TECHNIQUE['x_mitre_data_sources']:
            if DS not in ICS_DATA_SOURCES:
                ICS_DATA_SOURCES.append(DS)
ICS_DATA_SOURCES

['Alarm history',
 'Alarm thresholds',
 'Network protocol analysis',
 'Packet capture',
 'Sequential event recorder',
 'Data historian',
 'Netflow/Enclave netflow',
 'Authentication logs',
 'Windows event logs',
 'Web logs',
 'Web application firewall logs',
 'Application logs',
 'File monitoring',
 'Windows registry',
 'API monitoring',
 'File Monitoring',
 'Process monitoring',
 'Binary file metadata',
 'Controller program',
 'Network device logs',
 'Host network interfaces',
 'Process use of network',
 'Process command-line parameters',
 'Alarm History',
 'Sequential Event Recorder',
 'process use of network',
 'Web proxy',
 'SSl/TLS inspection',
 'Network intrusion detection system',
 'Windows error reporting',
 'Asset management',
 'Detonation chamber',
 'Digital signatures',
 'Windows Registry',
 'Data loss prevention',
 'Malware reverse engineering',
 'Controller parameters',
 'Anti-virus',
 'Third-party application logs',
 'Email gateway',
 'Mail server']

## Get All Groups from ICS ATT&CK

In [17]:
ICS_GROUPS = lift.get_ics_groups()
for GROUP in ICS_GROUPS:
    print(GROUP['name'])

TEMP.Veles
Dragonfly 2.0
HEXANE
Leafminer
APT33
OilRig
Dragonfly
Sandworm Team
Lazarus Group
ALLANITE


## Get All Malware from ICS ATT&CK

In [18]:
ICS_MALWARE = lift.get_ics_malware()
for MALWARE in ICS_MALWARE:
    print(MALWARE['name'])

Ryuk
LockerGoga
Stuxnet
VPNFilter
NotPetya
Triton
PLC-Blaster
WannaCry
Flame
Industroyer
Killdisk
Backdoor.Oldrea
BlackEnergy 3
ACAD/Medre.A
Conficker
Bad Rabbit
Duqu
